In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.metrics import jaccard_score
from sklearn.metrics import normalized_mutual_info_score
from scipy.stats import spearmanr

In [2]:
# Load Data 
ranked_data = pd.read_csv("../data/data_y_real.csv")
ranked_data['o_s'] = ranked_data['d_c'] + ranked_data['d_i']
outbreak_sizes = ranked_data.groupby(["seed"])["o_s"].agg("max")
new_df = pd.DataFrame()
new_df["seed"] = outbreak_sizes.index
# Get Full Outbreak Size
new_df["real_os"] = list(outbreak_sizes)
ranked_data = ranked_data.merge(new_df,left_on="seed",right_on="seed")
ranked_data = ranked_data[ranked_data["o_s"] > 0]
out_sizes = [ x for x in list(outbreak_sizes)  if x > 0 ]
# Count Number of times 
just_zero = ranked_data[ranked_data['o_s'] == 0].groupby(['seed']).size().rename('n_outbreaks')
zero_ids = list(just_zero.index)
good_seeds = []
## if an outbreak occurs - what is probability of detecting it for a given sentinel? 
g_zero = (ranked_data.groupby(["seed"])["real_os"].agg("max"))
ranked_data["norm_dt"] = ranked_data["d_t"] / 512
ranked_data["norm_di"] = ranked_data["d_i"] / ranked_data["real_os"]
sum_rank_df = ranked_data.groupby(["sent"])["d_f"].agg("sum")/1505

## Now, lets get the ranking list of the top 1%, 5%, and 10 % of nodes
## Then, find the jaccard similarity comparing the lists
## Let's also find the NMI

In [3]:
# Ensure numeric dtypes and drop invalid rows
for c in ["d_f", "d_i", "d_t"]:
    ranked_data[c] = pd.to_numeric(ranked_data[c], errors="coerce")
ranked_data = ranked_data.dropna(subset=["sent", "seed", "d_f", "d_i", "d_t"]).copy()

# ----------------------------------------------------
# Aggregate per sentinel
# ----------------------------------------------------
agg = ranked_data.groupby("sent", as_index=True).agg(
    dF_sum=("d_f", "sum"),     # total detections (higher better)
    dI_mean=("d_i", "mean"),   # mean infections at detection (lower better)
    dT_mean=("d_t", "mean"),   # mean time to detection (lower better)
)

# ----------------------------------------------------
# Compute ranks (1 = best)
# ----------------------------------------------------
agg["R_F"] = agg["dF_sum"].rank(ascending=False, method="min")
agg["R_I"] = agg["dI_mean"].rank(ascending=True,  method="min")
agg["R_T"] = agg["dT_mean"].rank(ascending=True,  method="min")

ranks = agg[["R_F", "R_I", "R_T"]]
n = len(ranks)

# ----------------------------------------------------
# Top-k thresholds
# ----------------------------------------------------
pct = {"top1": 0.01, "top5": 0.05, "top10": 0.10}
ks = {name: max(1, int(np.ceil(p * n))) for name, p in pct.items()}

def top_k_set(r: pd.Series, k: int) -> set:
    """Return indices with rank <= kth smallest rank (ties included)."""
    k = max(1, min(k, len(r)))
    kth = np.sort(np.asarray(r, dtype=float))[k - 1]
    return set(r.index[r <= kth])

top_sets = {
    m: {thr: top_k_set(ranks[m], k) for thr, k in ks.items()}
    for m in ["R_F", "R_I", "R_T"]
}

# ----------------------------------------------------
# Similarity metrics (Jaccard, NMI, Spearman)
# ----------------------------------------------------
def jaccard(a: set, b: set) -> float:
    return 1.0 if not a and not b else len(a & b) / max(1, len(a | b))

universe = ranks.index.sort_values()
def membership_series(members: set) -> pd.Series:
    s = pd.Series(0, index=universe, dtype=int)
    if members:
        s.loc[list(members)] = 1
    return s

pairs = [("R_F", "R_I"), ("R_F", "R_T"), ("R_I", "R_T")]
jaccard_results, nmi_results, corr_results = {}, {}, {}

for thr in pct.keys():
    jaccard_results[thr], nmi_results[thr], corr_results[thr] = {}, {}, {}
    for a, b in pairs:
        A, B = top_sets[a][thr], top_sets[b][thr]
        # Jaccard
        jaccard_results[thr][f"{a}_vs_{b}"] = jaccard(A, B)
        # NMI (binary membership)
        la, lb = membership_series(A), membership_series(B)
        nmi_results[thr][f"{a}_vs_{b}"] = normalized_mutual_info_score(la, lb)
        # Spearman correlation on full ranks (not just top-k)
        rho, _ = spearmanr(ranks[a], ranks[b])
        corr_results[thr][f"{a}_vs_{b}"] = float(rho)

# ----------------------------------------------------
# Output summary
# ----------------------------------------------------
print(f"Total sentinels: {n}")
print("k sizes:", ks)
for thr in pct.keys():
    print(f"\n=== Threshold: {thr} ===")
    print("Jaccard:")
    for k, v in jaccard_results[thr].items():
        print(f"  {k}: {v:.4f}")
    print("NMI (sklearn):")
    for k, v in nmi_results[thr].items():
        print(f"  {k}: {v:.4f}")
    print("Spearman ρ (full ranks):")
    for k, v in corr_results[thr].items():
        print(f"  {k}: {v:.4f}")

# Optional: keep useful objects
# agg, ranks, top_sets, jaccard_results, nmi_results, corr_results


Total sentinels: 1503
k sizes: {'top1': 16, 'top5': 76, 'top10': 151}

=== Threshold: top1 ===
Jaccard:
  R_F_vs_R_I: 0.0000
  R_F_vs_R_T: 0.0000
  R_I_vs_R_T: 0.7000
NMI (sklearn):
  R_F_vs_R_I: 0.0022
  R_F_vs_R_T: 0.0020
  R_I_vs_R_T: 0.6860
Spearman ρ (full ranks):
  R_F_vs_R_I: 0.7062
  R_F_vs_R_T: 0.8379
  R_I_vs_R_T: 0.9328

=== Threshold: top5 ===
Jaccard:
  R_F_vs_R_I: 0.2647
  R_F_vs_R_T: 0.4215
  R_I_vs_R_T: 0.6703
NMI (sklearn):
  R_F_vs_R_I: 0.1688
  R_F_vs_R_T: 0.3313
  R_I_vs_R_T: 0.5978
Spearman ρ (full ranks):
  R_F_vs_R_I: 0.7062
  R_F_vs_R_T: 0.8379
  R_I_vs_R_T: 0.9328

=== Threshold: top10 ===
Jaccard:
  R_F_vs_R_I: 0.4905
  R_F_vs_R_T: 0.6218
  R_I_vs_R_T: 0.7661
NMI (sklearn):
  R_F_vs_R_I: 0.3525
  R_F_vs_R_T: 0.4980
  R_I_vs_R_T: 0.6671
Spearman ρ (full ranks):
  R_F_vs_R_I: 0.7062
  R_F_vs_R_T: 0.8379
  R_I_vs_R_T: 0.9328


In [4]:
# correlation diagnostics
print("Spearman correlations on full ranks:")
print(ranks.corr(method="spearman"))

print("\nTop 10 sentinels by each metric:")
for col in ["R_F", "R_I", "R_T"]:
    print(f"\nTop 10 by {col}:")
    print(ranks[col].sort_values().head(10))


Spearman correlations on full ranks:
          R_F       R_I       R_T
R_F  1.000000  0.706242  0.837851
R_I  0.706242  1.000000  0.932847
R_T  0.837851  0.932847  1.000000

Top 10 sentinels by each metric:

Top 10 by R_F:
sent
8515     1.0
29744    2.0
9775     2.0
21695    4.0
30148    4.0
30069    4.0
29969    4.0
30073    4.0
30385    4.0
30361    4.0
Name: R_F, dtype: float64

Top 10 by R_I:
sent
22237    1.0
2496     1.0
8096     1.0
9940     1.0
24584    1.0
24590    1.0
8224     1.0
21125    1.0
9415     1.0
24499    1.0
Name: R_I, dtype: float64

Top 10 by R_T:
sent
21125     1.0
22672     2.0
24499     3.0
8096      4.0
24584     5.0
23092     6.0
9767      7.0
10448     8.0
2670      8.0
9940     10.0
Name: R_T, dtype: float64


In [5]:
# === Combine raw aggregates + ranks ===
rank_table = (
    agg[["dF_sum", "dI_mean", "dT_mean"]]
    .join(ranks[["R_F", "R_I", "R_T"]])
    .sort_values(["R_F", "R_I", "R_T"], ascending=[True, True, True])
)

# Peek
print("\nRank table (raw values + ranks):")
print(rank_table.head(20))

# Save to CSV (one file with everything)
rank_table.to_csv("sentinel_rank_table.csv", index=True)
print("\nSaved: sentinel_rank_table.csv")

# === If you want per-metric sorted views (top→bottom) ===
by_RF = rank_table.sort_values("R_F", ascending=True)
by_RI = rank_table.sort_values("R_I", ascending=True)
by_RT = rank_table.sort_values("R_T", ascending=True)

by_RF.head(20).to_csv("top_by_RF.csv")
by_RI.head(20).to_csv("top_by_RI.csv")
by_RT.head(20).to_csv("top_by_RT.csv")
print("Saved: top_by_RF.csv, top_by_RI.csv, top_by_RT.csv")

# === (Optional) add membership flags for top1/5/10 per metric ===
def add_membership_flags(df, ranks, ks, prefix):
    out = df.copy()
    for thr, k in ks.items():
        kth = np.sort(np.asarray(ranks, dtype=float))[k-1]
        mask = (ranks <= kth).astype(int)
        out[f"{prefix}_{thr}"] = mask
    return out

rank_table_flags = rank_table.copy()
rank_table_flags = add_membership_flags(rank_table_flags, ranks["R_F"], ks, "in_RF")
rank_table_flags = add_membership_flags(rank_table_flags, ranks["R_I"], ks, "in_RI")
rank_table_flags = add_membership_flags(rank_table_flags, ranks["R_T"], ks, "in_RT")

rank_table_flags.to_csv("sentinel_rank_table_with_flags.csv")
print("Saved: sentinel_rank_table_with_flags.csv")

# === (Optional) print top-k with raw values per threshold & metric ===
for thr, k in ks.items():
    print(f"\n--- Top {thr} by R_F (k={k}) ---")
    print(by_RF.head(k)[["dF_sum", "R_F"]])
    print(f"\n--- Top {thr} by R_I (k={k}) ---")
    print(by_RI.head(k)[["dI_mean", "R_I"]])
    print(f"\n--- Top {thr} by R_T (k={k}) ---")
    print(by_RT.head(k)[["dT_mean", "R_T"]])



Rank table (raw values + ranks):
       dF_sum     dI_mean     dT_mean   R_F    R_I    R_T
sent                                                     
8515     1472  765.804348  359.931386   1.0  920.0  927.0
9775     1450   48.410345  202.696552   2.0   35.0   22.0
29744    1450   52.608276  206.250345   2.0   36.0   23.0
30148    1449   54.147688  206.750863   4.0   37.0   24.0
30073    1449   58.104900  209.655625   4.0   38.0   25.0
21695    1449   59.175293  210.325052   4.0   39.0   26.0
29969    1449   71.125604  219.958592   4.0   46.0   31.0
30361    1449   71.306418  219.374741   4.0   47.0   30.0
30069    1449   78.020014  225.155280   4.0   48.0   39.0
30385    1449   83.803313  228.679779   4.0   53.0   42.0
1380     1448   97.517265  231.952348  11.0   60.0   49.0
1552     1448   99.859116  233.025552  11.0   65.0   52.0
8239     1447  223.554250  270.530062  13.0  182.0  163.0
30068    1445   88.043599  230.022145  14.0   54.0   44.0
33337    1445   89.731488  230.713495 

In [ ]:
# python3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# ranks: DataFrame with columns R_F, R_I, R_T (index = sentinels)

TOP_N = 100
SELECT_BY = "R_F"
OUTFILE = "rankings_3d_0_100_clipped.png"

# Select top-N by the chosen metric
k = min(TOP_N, len(ranks))
top_ids = ranks[SELECT_BY].nsmallest(k).index
top = ranks.loc[top_ids].copy()

# Count out-of-bounds (any axis > 100) among the top-N
oob_mask_any = (top[["R_T", "R_I", "R_F"]] > 100).any(axis=1)
n_oob_any = int(oob_mask_any.sum())

# (Optional) per-axis counts
n_oob_rt = int((top["R_T"] > 100).sum())
n_oob_ri = int((top["R_I"] > 100).sum())
n_oob_rf = int((top["R_F"] > 100).sum())

# Keep only in-bounds points for plotting (hard clip)
in_bounds = top.loc[~oob_mask_any].copy()

# Color by the selected ranking
colors = in_bounds[SELECT_BY].astype(float)

fig = plt.figure(figsize=(7.8, 6.4))
ax = fig.add_subplot(111, projection="3d")

sc = ax.scatter(
    in_bounds["R_T"].values,
    in_bounds["R_I"].values,
    in_bounds["R_F"].values,
    s=45,
    c=colors.values,
    cmap="viridis",
    depthshade=True,
    edgecolor="k",
    alpha=0.9,
)

ax.set_xlabel(f"$R_T$ (lower is better)")
ax.set_ylabel(f"$R_I$ (lower is better)")
ax.set_zlabel(f"$R_F$ (lower is better)")
ax.set_title(f"Top {k} by {SELECT_BY}: 3D Rank Scatter ($R_T$, $R_I$, $R_F$)")

# Show 0–100 window and invert (best corner near viewer)
ax.set_xlim(0, 100); ax.invert_xaxis()
ax.set_ylim(0, 100); ax.invert_yaxis()
ax.set_zlim(0, 100); ax.invert_zaxis()

# Colorbar + OOB counts
cbar = fig.colorbar(sc, ax=ax, pad=0.1, shrink=0.75)
cbar.set_label(f"{SELECT_BY} (rank value)", rotation=270, labelpad=15)
cbar.ax.text(
    0.5, -0.12,
    f"Top-{k} nodes with any rank > 100: {n_oob_any}\n"
    f"($R_T$>{100}: {n_oob_rt}, $R_I$>{100}: {n_oob_ri}, $R_F$>{100}: {n_oob_rf})",
    transform=cbar.ax.transAxes,
    ha="center", va="top", fontsize=9
)

plt.tight_layout()
plt.savefig(OUTFILE, dpi=300)
plt.close()
print(f"Saved 3D plot → {OUTFILE} (plotted {len(in_bounds)} in-bounds points; "
      f"hidden {n_oob_any} out-of-bounds)")
